In [1]:
import sys
import json
import cobra
import cplex
import cobrakbase
import escher
from escher import Builder
#Put the path to ModelSEEDpy on your machine here
sys.path.append("/Users/chenry/code/ModelSEEDpy")
#import modelseedpy.fbapkg
from modelseedpy import MSPackageManager

cobrakbase 0.2.7


In [2]:
file = open('plant_map.json',)
plantmap = file.read()
builder = Builder(map_json=plantmap)
builder.reaction_scale=[
        { "type": 'value', "value": 0, "color": '#dcdcdc', "size": 10 },
        { "type": 'value', "value": 0.000001, "color": '#9696ff', "size": 25 },
        { "type": 'value', "value": 1, "color": '#ff0000', "size": 35 },
        { "type": 'value', "value": 20, "color": '#209123', "size": 50 },
    ]
builder

Builder(reaction_scale=[{'type': 'value', 'value': 0, 'color': '#dcdcdc', 'size': 10}, {'type': 'value', 'valu…

In [3]:
kbase_api = cobrakbase.KBaseAPI()
model = kbase_api.get_from_ws("C4_Calvin_Zmays_Model",25427)
model.solver = 'optlang-cplex'
#Setting model to complete media
#media = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
pkgmgr = MSPackageManager.get_pkg_mgr(model,1)
pkgmgr.getpkg("KBaseMediaPkg",1).build_package(None)

#Reading reaction protein abundances
data = ""
with open('/Users/chenry/code/ModelSEEDpy/examples/ReactionProtein.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_proteins = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        for i in range(2, len(array)):
            if headers[i] not in reaction_proteins:
                reaction_proteins[headers[i]] = dict()
            reaction_proteins[headers[i]][array[0]] = abs(float(array[i]))

#Reading reaction kcat values
data = ""
with open('/Users/chenry/code/ModelSEEDpy/examples/KCats.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_kcat = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        reaction_kcat[array[0]] = abs(float(array[1]))

#Reading measured reaction fluxes
data = ""
with open('/Users/chenry/code/ModelSEEDpy/examples/MeasuredReaction.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_measures = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        if array[1] not in reaction_measures:
                reaction_measures[array[1]] = dict()
        rxnid = array[0]
        if rxnid[-1:] != "1":
            rxnid += "0"
        reaction_measures[array[1]][rxnid] = abs(float(array[2]))

#Applying flexible biomass constraints
protein = 0.0948
protein_flux = -1*(0.2-protein)
pkgmgr.getpkg("FlexibleBiomassPkg",1).build_package({"bio_rxn_id":"bio1","use_rna_class":[-0.75,0.75],
    "use_dna_class":[-0.75,0.75],
    "use_protein_class":[protein_flux,protein_flux],
    "use_energy_class":[-0.1,0.1]})

with model:
    #Applying proteome fitting constraints
    pkgmgr.getpkg("ProteomeFittingPkg",1).build_package({
        "rxn_proteome":reaction_proteins["Mature_leaf"],
        "flux_values":reaction_measures["Mature_leaf"],
        "kcat_values":reaction_kcat,
        "prot_coef" : protein,#Set to the fraction of the cell that is protein
        "totalflux" : 1,#Set to one if we're fitting flux magnitude rather than actual flux to flux measurements
        "kcat_coef" : 0.02,#kapp = kcat_coef * kcat
        "obj_kfit":10,#Coefficient on kcat fitting terms in objective function
        "obj_kvfit":1,#Coefficient on kinetic expression fitting terms in objective function
        "obj_vfit":1000,#Coefficient on flux measurement terms in objective function
        "set_objective":1
    })
    
    #Constraining biomass flux to reference value
    rxn = model.reactions.get_by_id("bio1")
    rxn.upper_bound = 0.006395833
    rxn.lower_bound = 0.006395833
    
    #Printing LP file
    with open('SingleTissueProteomeData.lp', 'w') as out:
        out.write(str(model.solver))
    
    #Optimizing
    sol=model.optimize()
    model.summary()
    builder.model = model
    builder.reaction_data = sol.fluxes
    #Print solution:
    #Key variables:
    #rxn#####_c0_kapp - k apparent for reaction 29
    #rxn#####_c0_kvfit - fit of reaction flux to equation kapp*[protein]
    #rxn#####_c0_kfit - fit of kapp to 0.02*kcat
    #rxn#####_c0_tf - total flux through reaction
    #rxn#####_c0_vfit - fit to measured value for this reaction
    #Objective value is to minimize the sum of squares for kvfit, kfit, and vfit variables

In [4]:
tissues = [
    "Zone_1","Zone_2","Zone_3","GermEmbryo_2_DAI","Pericarp_Aleurone_27_DAP",
    "Endosperm_Crown_27_DAP","Endosperm_12_DAP","Endosperm_10_DAP","Endosperm_8_DAP","Embryo_38_DAP",
    "Embryo_20_DAP","root_Stele","root_Cortex","root_EZ","root_MZ"
]

biomass_fluxes = {
    "root_MZ" : 0.049925,
    "root_EZ" : 0.049925,
    "root_Cortex" : 0.049925,
    "root_Stele" : 0.049925,
    "Embryo_20_DAP" : 0.015591667,
    "Embryo_38_DAP" : 0.015591667,
    "Endosperm_8_DAP" : 0.010208333,
    "Endosperm_10_DAP" : 0.010208333,
    "Endosperm_12_DAP" : 0.010208333,
    "Endosperm_Crown_27_DAP" : 0.010208333,
    "Pericarp_Aleurone_27_DAP" : 0.001529167,
    "GermEmbryo_2_DAI" : 0.015591667,
    "Zone_1" : 0.006395833,
    "Zone_2" : 0.006395833,
    "Zone_3" : 0.006395833,
    "Mature_leaf" : 0.006395833
}

protein_fraction = {
    "root_MZ" : 0,
    "root_EZ" : 0,
    "root_Cortex" : 0,
    "root_Stele" : 0,
    "Embryo_20_DAP" : 0,
    "Embryo_38_DAP" : 0,
    "Endosperm_8_DAP" : 0,
    "Endosperm_10_DAP" : 0,
    "Endosperm_12_DAP" : 0,
    "Endosperm_Crown_27_DAP" : 0,
    "Pericarp_Aleurone_27_DAP" : 0,
    "GermEmbryo_2_DAI" : 0,
    "Zone_1" : 0,
    "Zone_2" : 0,
    "Zone_3" : 0,
    "Mature_leaf" : 0.0948
}

modellist = []
for tissue in tissues:
    #Safe cloning method
    clone_model = cobra.io.json.from_json(cobra.io.json.to_json(model))
    #Adding tissue specific protein constraints
    pmp = ProteomeFittingPkg(clone_model)
    pmp.build_package({
        "rxn_proteome":reaction_proteins[tissue],
        "flux_values":reaction_measures[tissue],
        "kcat_values":reaction_kcat,
        "prot_coef" : protein_fraction[tissue],#Set to the fraction of the cell that is protein
        "totalflux" : 1,#Set to one if we're fitting flux magnitude rather than actual flux to flux measurements
        "kcat_coef" : 0.02,#kapp = kcat_coef * kcat
        "obj_kfit":10,#Coefficient on kcat fitting terms in objective function
        "obj_kvfit":1,#Coefficient on kinetic expression fitting terms in objective function
        "obj_vfit":1000,#Coefficient on flux measurement terms in objective function
        "set_objective":1
    })
    #Constraining biomass flux to reference value
    rxn = clone_model.reactions.get_by_id("bio1")
    rxn.upper_bound = biomass_fluxes[tissue]
    rxn.lower_bound = biomass_fluxes[tissue]
    modellist.append(clone_model)

#Adding leaf proteome constraints
pmp = ProteomeFittingPkg(model)
pmp.build_package({
    "rxn_proteome":reaction_proteins["Mature_leaf"],
    "flux_values":reaction_measures["Mature_leaf"],
    "kcat_values":reaction_kcat,
    "prot_coef" : 0.0948,#Set to the fraction of the cell that is protein
    "totalflux" : 1,#Set to one if we're fitting flux magnitude rather than actual flux to flux measurements
    "kcat_coef" : 0.02,#kapp = kcat_coef * kcat
    "obj_kfit":1,#Coefficient on kcat fitting terms in objective function
    "obj_kvfit":1,#Coefficient on kinetic expression fitting terms in objective function
    "obj_vfit":1,#Coefficient on flux measurement terms in objective function
    "set_objective":1
})
    
#Merging all models together
pmp = ProblemReplicationPkg(model)
pmp.build_package({"models":modellist,"shared_variable_packages":pmp:["kapp"]})
 
sol=model.optimize()
model.summary()  

#Printing LP file
with open('MultiTissueProteomeData.lp', 'w') as out:
    out.write(str(model.solver))

SyntaxError: invalid syntax (<ipython-input-4-7d31fd826102>, line 86)